<a href="https://colab.research.google.com/github/gustavoiesb/dataset_files/blob/master/Gera%C3%A7%C3%A3o_de_texto_caractere_por_caractere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random
import io

## Preparando os dados


In [2]:
caminho = keras.utils.get_file(
    "memorias_de_um_sargento_de_milicias.txt", 
    origin="https://raw.githubusercontent.com/gustavoiesb/dataset_files/master/memorias_de_um_sargento_de_milicias.txt"
)

In [3]:
with io.open(caminho, encoding="utf-8") as f:
    texto = f.read().lower()

# Removendo quebras de linhas
texto = texto.replace("\n", " ") 

print("Tamanho do texto (length):", len(texto))

Tamanho do texto (length): 341896


In [4]:
chars = sorted(list(set(texto)))
print("Quantidade de caracteres:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Criando as sequências par entrada no modelo
maxlen = 40
step = 3
sentencas = []
next_chars = []
for i in range(0, len(texto) - maxlen, step):
    sentencas.append(texto[i : i + maxlen])
    next_chars.append(texto[i + maxlen])

print("Quantidade de sequências:", len(sentencas))

Quantidade de caracteres: 60
Quantidade de sequências: 113952


In [5]:
x = np.zeros((len(sentencas), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentencas), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentencas):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

## Criação do modelo com apenas uma camada LSTM


In [6]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

### ***Por que usar amostragem para gerar texto a partir de um modelo de linguagem RNN treinado ?***

Depois de treinar um modelo de linguagem, muitas vezes você gostaria de usar o modelo para gerar um novo texto. Para um modelo RNN para geração de palvras, o texto é gerado uma palavra por vez. Em cada etapa, o modelo gera uma distribuição de probabilidade sobre todo o vocabulário. Com um vocabulário de exemplo/teste, como:

$$vocabulário = \begin{bmatrix}
gato\\
cão\\
sapo
\end{bmatrix}$$

A saída da distribuição pelo modelo em cada etapa de geração pode ficar assim:

$$previsões = \begin{bmatrix}
0,7\\
0,1\\
0,2
\end{bmatrix}$$

Cada entrada no vetor corresponde a uma entrada em nosso vocabulário/conjunto de palavras.

Depois disso, cabe ao usuário selecionar uma palavra desta distribuição como a próxima palavra no texto gerado. Dois métodos muito diferentes para escolher uma palavra da distribuição são:

**Greedy Search (pesquisa gananciosa)**: pegará a palavra com a maior probabilidade (escolherá "gato" nesse caso)

**Amostragem:** amostra da distribuição, leva em consideração as probabilidades (escolherá uma palavra aleatória, mas "gato" tem a maior chance de ser escolhido)

Portanto, a pesquisa retornará uma palavra(caractere) muito provável (e a mesma palavra(caractere) toda vez que for requisitado), enquanto a amostragem retornará uma palavra/caractere das palavras/caracteres mais provavéis, porém um texto mais variado. Geralmente, isso significa que a amostragem torna a geração de textos mais interessante.

## Função responsável por gerar uma amostragem 

> Retornando um indice de acordo com a distribuição de probabilidades

Explicação da necessidade de amostragem de textos em Redes Neurais Recorrentes (RNN) : https://datascience.stackexchange.com/questions/72770/why-we-sample-when-predicting-with-recurent-neural-network


In [7]:
def gerar_idx_amostra(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Treinamento do modelo e geração dos textos


In [8]:
epocas = 40
batch_size = 128

for epoca in range(epocas):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Gerando texto na época: %d" % epoca)

    indice_inicial = random.randint(0, len(texto) - maxlen - 1)
    for diversidade in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversidade:", diversidade)

        gerado = ""
        setenca = texto[indice_inicial : indice_inicial + maxlen]
        print('...Gerando com a semente (sentença): "' + setenca + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(setenca):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            proximo_indice = gerar_idx_amostra(preds, diversidade)
            proximo_caracter = indices_char[proximo_indice]
            setenca = setenca[1:] + proximo_caracter
            gerado += proximo_caracter

        print('...Geração finalizada: \n', gerado)

891/891 [==============================] - 114s 126ms/step - loss: 2.3110

Gerando texto na época: 0
...Diversidade: 0.2
...Gerando com a semente (sentença): "nda em conta as inocentes caçoadas que a"
...Geração finalizada: 
  como de dia por que a para de ou dia por que se por interrado do leonardo por que se se por que o tem por que de de por que a para de ou bom o por que o padre de para de contrado de os porto de ou porto de ou risa de ontro de os porto de o como de ou nos o porto de ou disto de os porto de ontro de ou dia por que no mesmo de ou dia por que disso de ontro de o propessado de por que se por que disse
...Diversidade: 0.5
...Gerando com a semente (sentença): "nda em conta as inocentes caçoadas que a"
...Geração finalizada: 
  se estara de uma entravam o perquer ques de onde resta de fio de capatada a sua sua o leonardo vidos em cora d. maria, que porto de como de ou disto de lisaira a desto de poré a seu por o menino de osta de por que de alguma para de nem dia como de

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


...Geração finalizada: 
 sidade. as velhas de seu padrinho de trabalhos a comadre de tal era a comadre de alguém que tinha de contentada e contra ele, e apenas algumas soldadas por cabeça de contente de uma vida e de destante de alturas para o compadre de cada um dos passos de seu partido do compadre com o leonardo de escontada a comadre de contentar a comadre por isso desse de todas as de parecer. a comadre, disse a coma
...Diversidade: 0.5
...Gerando com a semente (sentença): "sam estas cenas instituições muito curio"
...Geração finalizada: 
 sidade de cadeiras e pertoras, o camo de compadre achou-se a ppuga por causa do leonardo-pataca, e preondeu de visto de precirou-se em que disse um companhio de ordentas, e entre a cigana, e a comadre de alturas para contrado e cuja comadre foi por ele e desafirar dos olhos de uma maria da causa de ser contra e cabelos de medo, e estava ele e mangava que se achava o que se achava de todo algum est
...Diversidade: 1.0
...Gerando com a semente (se